In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from use_case.baseline import * 
from tests.eval import *

payoff_i = np.random.uniform(-10, 10, (5, 5))
payoff_j = np.transpose(payoff_i).copy()

# Initialize environment
N_ACTIONS = payoff_i.shape[0]
N_AGENTS = 1000
env = BaselineEnvironment(N_AGENTS, payoff_i, payoff_j, total_games = 5)

# Actual Run

In [36]:
from models.model import *
from models.trainer import *


In [37]:
# Configure the network here
parameters = ParameterSettings(
    n_agents = N_AGENTS,
    d_action = N_ACTIONS, 
    d_obs = env.obs_size, 
    d_traits = 1,
    d_beliefs = 1
)
parameters.device = "cuda" if torch.cuda.is_available() else "cpu"

model = Model(parameters)

In [38]:
equilibriua = find_pure_equilibria(payoff_i, payoff_j)

for eq in equilibriua:
    x, y = eq 
    a = (y[0] + y[1]) / 2

    print(x, a)

(2, 2) 8.72111006557343


In [39]:
evaluate_policy(model, env, 10)

Average Return: -0.3585144560368139
Total returns: -3.585144560368139
Action Distribution
[0 1 2 3 4]
[260 200 192 207 141]


In [ ]:
# Setup the training loop
training_parameters = TrainingParameters(
    actor_training_loops = 1000, 
    outer_loops = 100,
    learning_rate= 0.01,
    experience_buffer_size = 3,
    entropy_coeff= 0
)

train_model(model, env, training_parameters)
        

Epoch 0


Actor Training: 100%|██████████| 4/4 [00:00<00:00,  5.95it/s]


Average Return: -0.40325079056516433
Total returns: -4.032507905651643
Action Distribution
[0 1 2 3 4]
[140  28 242 289 301]
Epoch 1


Actor Training: 100%|██████████| 4/4 [00:00<00:00,  7.31it/s]


Average Return: 1.0191109646606475
Total returns: 10.191109646606476
Action Distribution
[0 1 2 3 4]
[167  20 317 246 250]
Epoch 2


...

Average Return: 3.9739539104362342
Total returns: 39.73953910436234
Action Distribution
[0 1 2 3 4]
[125  22 407 193 253]
Epoch 10


Actor Training: 100%|██████████| 4/4 [00:00<00:00,  7.53it/s]


Average Return: 3.8310024924479125
Total returns: 38.31002492447912
Action Distribution
[0 1 2 3 4]
[122  28 368 216 266]
Epoch 11


Actor Training:  25%|██▌       | 1/4 [00:00<00:00,  5.12it/s]


KeyboardInterrupt: 